## Segmenting and Clustering
_adding latitude and longitude data_

In [155]:
!conda install beautifulsoup4 --yes

Solving environment: done

# All requested packages already installed.



In [156]:
!pip install geocoder

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find('table', class_='wikitable sortable')
tbody = table.find('tbody')
trs = tbody.find_all('tr')


In [159]:
postal_codes = []
boroughs = []
neighborhoods = []

td_i = 0;

for tr in trs:
    tds = tr.find_all('td')
    
    postal_code = ''
    borough = ''
    neighborhood = ''
    
    for td in tds:
        
        if td_i == 0:
            postal_code = td.text.rstrip('\n')
        elif td_i == 1:
            borough = td.text.rstrip('\n')
        elif td_i == 2:
            neighborhood = td.text.rstrip('\n')
            
        td_i = td_i + 1
        
    if borough != 'Not assigned' and borough != '':
            postal_codes.append(postal_code)
            boroughs.append(borough)
            neighborhoods.append(neighborhood)
        
    td_i = 0
    
np_data = np.array([postal_codes, boroughs, neighborhoods]).transpose()
df = pd.DataFrame(data=np_data, columns=['Postal Code', 'Borough', 'Neighborhood'])
df.shape

(103, 3)

In [160]:
lldf = pd.read_csv('http://cocl.us/Geospatial_data')

In [161]:
df['Latitude'] = 0
df['Longitude'] = 0
df = df.astype({'Latitude': float, 'Longitude': float})
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,0.0,0.0
1,M4A,North York,Victoria Village,0.0,0.0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0.0,0.0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",0.0,0.0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0.0,0.0


In [162]:
for index, row in df.iterrows():
    
    lon_lat_r = lldf.loc[lldf['Postal Code'] == row['Postal Code']]
    df.loc[index, 'Latitude'] = lon_lat_r[['Latitude']].iloc[0].values[0]
    df.loc[index, 'Longitude'] = lon_lat_r[['Longitude']].iloc[0].values[0]
    

In [163]:
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
